In [1]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [2]:
!pip install keras-rl
!pip install keras==2.2.4
!pip install tensorflow-gpu==1.13.1 

     |████████████████████████████████| 40kB 2.3MB/s 
  Created wheel for keras-rl: filename=keras_rl-0.4.2-cp36-none-any.whl size=48379 sha256=674bf43e29b61b730e1cf879ba85d166676467f38d603666cd933de877831d15
  Stored in directory: /root/.cache/pip/wheels/7d/4d/84/9254c9f2e8f51865cb0dac8e79da85330c735551d31f73c894
Successfully built keras-rl
     |████████████████████████████████| 317kB 2.8MB/s 
  Found existing installation: Keras 2.2.5
    Uninstalling Keras-2.2.5:
      Successfully uninstalled Keras-2.2.5
     |████████████████████████████████| 345.2MB 49kB/s 
     |████████████████████████████████| 368kB 38.0MB/s 
     |████████████████████████████████| 3.2MB 29.8MB/s 
ERROR: tensorflow 1.15.0 has requirement tensorboard<1.16.0,>=1.15.0, but you'll have tensorboard 1.13.1 which is incompatible.
ERROR: tensorflow 1.15.0 has requirement tensorflow-estimator==1.15.1, but you'll have tensorflow-estimator 1.13.0 which is incompatible.
  Found existing installation: tensorflow-estimator

In [1]:
from __future__ import division
import gym
import atari_py
from PIL import Image
import numpy as np
import keras
import tensorflow as tf

from keras.models import Sequential, Model
from keras.layers import Dense, Activation, Flatten, Convolution2D, Permute, Input, concatenate
from keras.optimizers import Adam
from keras.utils import plot_model
import keras.backend as K

from rl.agents.dqn import DQNAgent
from rl.policy import LinearAnnealedPolicy, BoltzmannQPolicy, EpsGreedyQPolicy
from rl.memory import SequentialMemory
from rl.core import Processor

print(atari_py.list_games())

print(keras.__version__)
print(tf.__version__)


Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:529: FutureWarn

['kaboom', 'time_pilot', 'chopper_command', 'asterix', 'ms_pacman', 'adventure', 'elevator_action', 'kung_fu_master', 'robotank', 'boxing', 'jamesbond', 'centipede', 'journey_escape', 'riverraid', 'defender', 'pooyan', 'crazy_climber', 'zaxxon', 'pong', 'amidar', 'wizard_of_wor', 'fishing_derby', 'gravitar', 'kangaroo', 'road_runner', 'bank_heist', 'gopher', 'battle_zone', 'hero', 'star_gunner', 'qbert', 'breakout', 'space_invaders', 'phoenix', 'solaris', 'tennis', 'asteroids', 'frostbite', 'assault', 'double_dunk', 'alien', 'seaquest', 'carnival', 'demon_attack', 'atlantis', 'up_n_down', 'ice_hockey', 'air_raid', 'freeway', 'krull', 'private_eye', 'venture', 'beam_rider', 'pitfall', 'name_this_game', 'video_pinball', 'bowling', 'yars_revenge', 'berzerk', 'tutankham', 'montezuma_revenge', 'skiing', 'enduro']
2.2.4
1.13.1


In [0]:
INPUT_SHAPE = (84, 84)
WINDOW_LENGTH = 4

class AtariProcessor(Processor):
    def process_observation(self, observation):
        assert observation.ndim == 3  # (height, width, channel)
        img = Image.fromarray(observation)
        img = img.resize(INPUT_SHAPE).convert('L')  # resize and convert to grayscale
        processed_observation = np.array(img)
        assert processed_observation.shape == INPUT_SHAPE
        return processed_observation.astype('float32')/255 

    #def process_state_batch(self, batch):
     #   processed_batch = batch.astype('float32') / 255.
      #  return processed_batch

    #def process_reward(self, reward):
     #   return np.clip(reward, -1., 1.)

In [0]:
#game = 'BeamRider-v0' #9
#game = 'Breakout-v0' #4
#game = 'Solaris-v0' #18
#game = 'DemonAttack-v0' #6
#game = 'Centipede-v0' #18

game1 = 'AirRaid-v0' #6
game2 = 'BankHeist-v0' #18


env1 = gym.make(game1, full_action_space=True)
np.random.seed(123)
env1.seed(123)

env2 = gym.make(game2, full_action_space=True)
np.random.seed(123)
env2.seed(123)


if (env1.action_space.n == env2.action_space.n):
  nb_actions = env1.action_space.n
else:
  raise RuntimeError('Actions does not match for 2 enviroments.')

In [4]:
input_shape = (WINDOW_LENGTH,) + INPUT_SHAPE


# CNN for environment 1
env1_input_layer = Input(shape=input_shape)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    env1_permute = Permute((2, 3, 1))(env1_input_layer)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    env1_permute = Permute((1, 2, 3))(env1_input_layer)
else:
    raise RuntimeError('Unknown image_dim_ordering for environment 1.')

env1_Conv_1 = Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu')(env1_permute)
env1_Conv_2 = Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu')(env1_Conv_1)
env1_Conv_3 = Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu')(env1_Conv_2)
env1_flatten = Flatten()(env1_Conv_3)
env1_dense = Dense(512, activation = 'relu')(env1_flatten)


# CNN for environment 2
env2_input_layer = Input(shape=input_shape)
if K.image_dim_ordering() == 'tf':
    # (width, height, channels)
    env2_permute = Permute((2, 3, 1))(env2_input_layer)
elif K.image_dim_ordering() == 'th':
    # (channels, width, height)
    env2_permute = Permute((1, 2, 3))(env2_input_layer)
else:
    raise RuntimeError('Unknown image_dim_ordering for environment 2.')
    
env2_Conv_1 = Convolution2D(32, (8, 8), strides=(4, 4), activation = 'relu')(env2_permute)
env2_Conv_2 = Convolution2D(64, (4, 4), strides=(2, 2), activation = 'relu')(env2_Conv_1)
env2_Conv_3 = Convolution2D(64, (3, 3), strides=(1, 1), activation = 'relu')(env2_Conv_2)
env2_flatten = Flatten()(env2_Conv_3)
env2_dense = Dense(512, activation = 'relu')(env2_flatten)

shared_layer = Dense(nb_actions, activation = 'linear', name = 'shared_layer')

env1_output = shared_layer(env1_dense)
env2_output = shared_layer(env2_dense)


model1 = Model(inputs = [env1_input_layer], outputs = [env1_output]) 
model2 =  Model(inputs = [env2_input_layer], outputs = [env2_output])

print(model1.summary())
plot_model(model1)
print(model2.summary())
plot_model(model2)


Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 4, 84, 84)         0         
_________________________________________________________________
permute_1 (Permute)          (None, 84, 84, 4)         0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 20, 20, 32)        8224      
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 9, 9, 64)          32832     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 7, 7, 64)          36928     
_________________________________________________________________
flatten_1 (Flatten)          (None, 3136)              0         
_________________________________________________________________
dens

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000000)

dqn1 = DQNAgent(model=model1, nb_actions=nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn1.compile(Adam(lr=.00025), metrics=['mae'])


In [7]:
dqn1.fit(env1, nb_steps=300000, visualize=False, verbose=1) 
dqn1.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_dissimilar',overwrite=True)

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 0.8500
13 episodes - episode_reward: 650.000 [150.000, 1900.000] - ale.lives: 0.999

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: 0.8475
10 episodes - episode_reward: 597.500 [200.000, 1275.000] - ale.lives: 0.999

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: 0.6050
12 episodes - episode_reward: 712.500 [25.000, 3325.000] - ale.lives: 0.999

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 25s 3ms/step - reward: 0.7075
13 episodes - episode_reward: 440.385 [200.000, 725.000] - ale.lives: 0.999

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 25s 2ms/step - reward: 0.6725
12 episodes - episode_reward: 643.750 [225.000, 1800.000] - ale.lives: 0.999

Interval 6 (50000 steps pe

In [8]:
#dqn1.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_dissimilar')
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 150.000, steps: 305
Episode 2: reward: 0.000, steps: 287
Episode 3: reward: 150.000, steps: 1028
Episode 4: reward: 450.000, steps: 514
Episode 5: reward: 0.000, steps: 266
Episode 6: reward: 200.000, steps: 332
Episode 7: reward: 200.000, steps: 602
Episode 8: reward: 275.000, steps: 426
Episode 9: reward: 75.000, steps: 233
Episode 10: reward: 425.000, steps: 815


In [9]:
import matplotlib.pyplot as plt
import seaborn as sns

shared_weights, shared_bias = model1.get_layer('shared_layer').weights
sns.distplot(K.eval(shared_weights).ravel())
plt.savefig('/content/drive/My Drive/Colab Notebooks/Game Data/shared_weights_before_dissimilar.jpg')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
memory = SequentialMemory(limit=1000000, window_length=WINDOW_LENGTH)
processor = AtariProcessor()
policy = LinearAnnealedPolicy(EpsGreedyQPolicy(), attr='eps', value_max=1., value_min=.1, value_test=.05, nb_steps=1000000)

dqn2 = DQNAgent(model=model2, nb_actions=nb_actions, policy=policy, memory=memory, processor=processor, nb_steps_warmup=50000, target_model_update=10000,train_interval=4)
dqn2.compile(Adam(lr=.00025), metrics=['mae'])


In [11]:
dqn2.fit(env2, nb_steps=300000, visualize=False, verbose=1)
dqn2.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model2_weights_dissimilar',overwrite=True)

Training for 300000 steps ...
Interval 1 (0 steps performed)
10000/10000 [==============================] - 30s 3ms/step - reward: 0.0270
14 episodes - episode_reward: 17.857 [0.000, 30.000] - ale.lives: 1.995

Interval 2 (10000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 0.0260
15 episodes - episode_reward: 17.333 [0.000, 30.000] - ale.lives: 2.084

Interval 3 (20000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 0.0260
14 episodes - episode_reward: 19.286 [0.000, 40.000] - ale.lives: 1.936

Interval 4 (30000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 0.0320
15 episodes - episode_reward: 22.000 [0.000, 70.000] - ale.lives: 2.022

Interval 5 (40000 steps performed)
10000/10000 [==============================] - 29s 3ms/step - reward: 0.0290
14 episodes - episode_reward: 19.286 [0.000, 40.000] - ale.lives: 1.982

Interval 6 (50000 steps performed)
10000/10000 [=

In [12]:
dqn2.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model2_weights_dissimilar')
dqn2.test(env2, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 170.000, steps: 938
Episode 2: reward: 80.000, steps: 626
Episode 3: reward: 140.000, steps: 737
Episode 4: reward: 140.000, steps: 791
Episode 5: reward: 120.000, steps: 636
Episode 6: reward: 160.000, steps: 786
Episode 7: reward: 190.000, steps: 788
Episode 8: reward: 60.000, steps: 538
Episode 9: reward: 170.000, steps: 892
Episode 10: reward: 50.000, steps: 468


In [13]:
shared_weights, shared_bias = model1.get_layer('shared_layer').weights
sns.distplot(K.eval(shared_weights).ravel())
plt.savefig('/content/drive/My Drive/Colab Notebooks/Game Data/shared_weights_after_dissimilar.jpg')
plt.clf()

<Figure size 432x288 with 0 Axes>

In [0]:
dqn1.save_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_dissimilar_changed',overwrite=True)

In [15]:
dqn1.load_weights('/content/drive/My Drive/Colab Notebooks/Game Data/DQN_model1_weights_dissimilar_changed')
dqn1.test(env1, nb_episodes=10, visualize=False)

Testing for 10 episodes ...
Episode 1: reward: 650.000, steps: 1149
Episode 2: reward: 825.000, steps: 984
Episode 3: reward: 900.000, steps: 1483
Episode 4: reward: 1325.000, steps: 2152
Episode 5: reward: 1000.000, steps: 1574
Episode 6: reward: 575.000, steps: 847
Episode 7: reward: 2000.000, steps: 2728
Episode 8: reward: 75.000, steps: 191
Episode 9: reward: 975.000, steps: 1452
Episode 10: reward: 775.000, steps: 1392


In [0]:
#env.close()